In [ ]:
import numpy as np
import numpy.linalg

In [ ]:
import data_receiver
from mathlib import *
from plotlib import *

In [ ]:
%matplotlib widget
np.set_printoptions(precision=5, suppress=True)

In [ ]:
# sampling rate
dt = 0.01    # s

# the initialization interval
ts = 1    # s

# pull data from phone
data order: gyroscorpe, accelerometer, magnetometer

In [ ]:
r = data_receiver.Receiver()

data = []

for line in r.receive():
    data.append(line.split(','))

data = np.array(data, dtype = np.float)

## Initialization

In [ ]:
# discard the first and last few readings
# for some reason they fluctuate a lot
w = data[10:-10, 0:3]
a = data[10:-10, 3:6]
m = data[10:-10, 6:9]

if(np.shape(w)[0] < ts/dt):
    print("not enough data for intialization!")

# gravity
gn = a[:int(ts/dt)].mean(axis = 0)
gn = -gn[:, np.newaxis]
g0 = np.linalg.norm(gn)  # save the initial magnitude of gravity

# magnetic field
mn = m[:int(ts/dt)].mean(axis = 0)
mn = Normalized(mn)[:, np.newaxis]  # magnitude is not important

# cut the initialization data
w = w[int(ts/dt) - 1:] - w[:int(ts/dt)].mean(axis=0)
a = a[int(ts/dt):]
m = m[int(ts/dt):]

sample_number = np.shape(a)[0]

In [ ]:
a_filtered, w_filtered, m_filtered = Filt_signal((a, w, m), dt=dt, wn=10, btype='lowpass')
plot_signal([a, a_filtered], [w, w_filtered], [m, m_filtered])

## Kalman Filter

In [ ]:
gyro_noise = 1e-4
acc_noise = 1e-2
mag_noise = 1e-2

P = 1e-8 * I(4)

In [ ]:
a_nav = []

q = np.array([[1, 0, 0, 0]]).T

t = 0
while t < sample_number:
    wt = w_filtered[t, np.newaxis].T
    at = a_filtered[t, np.newaxis].T
    mt = m_filtered[t, np.newaxis].T 
    mt = Normalized(mt)

    # Propagation
    Ft = F(q, wt, dt)
    Gt = G(q)
    Q = (gyro_noise * dt)**2 * Gt @ Gt.T
    
    q = Ft @ q
    q = Normalized(q)
    P = Ft @ P @ Ft.T + Q    

    # Measurement Update
    # Use only normalized measurements to reduce error!
    
    # acc and mag prediction
    pa = Normalized(-Rotate(q) @ gn)
    pm = Normalized(Rotate(q) @ mn)

    # Residual
    Eps = np.vstack((Normalized(at), mt)) - np.vstack((pa, pm))
    
    # internal error + external error
    Ra = [(acc_noise / np.linalg.norm(at))**2 + (1 - g0 / np.linalg.norm(at))**2] * 3
    Rm = [mag_noise**2] * 3
    R = np.diag(Ra + Rm)
    
    Ht = H(q, gn, mn)

    S = Ht @ P @ Ht.T + R
    K = P @ Ht.T @ np.linalg.inv(S)
    q = q + K @ Eps
    P = P - K @ Ht @ P
    
    # Post Correction
    q = Normalized(q)
    P = 0.5 * (P + P.T)  # make sure P is symmertical
    
    tmp = -I(4)
    tmp[0, 0] = 1
    an = Rotate(tmp @ q) @ at + gn

    a_nav.append(an.T[0])
    
    t += 1

a_nav = np.array(a_nav)

In [ ]:
filtered_a_nav, = Filt_signal([a_nav], dt=dt, wn=0.5, btype='highpass')
plot_3([a_nav, filtered_a_nav])

In [ ]:
velocities = []
v = np.zeros((3, 1))

t1 = 0
vt1 = np.zeros((3, 1))

t = 0
while t < sample_number:
    at = filtered_a_nav[t, np.newaxis].T

    if np.linalg.norm(at) > 1e-1 * g0:
        v = v + at * dt
    else:
        drift_rate = (v - vt1) / (t - t1)

        for idx in range(len(velocities[t1:])):
            velocities[t1 + idx] = (velocities[t1 + idx] - idx * drift_rate.T)[0]

        t1 = t
        vt1 = v
        v = np.zeros((3, 1))

    velocities.append(v.T[0])

    t += 1

velocities = np.array(velocities)

In [ ]:
filtered_velocities, = Filt_signal([velocities], dt=dt, wn=1e-1, btype='highpass')
plot_3([velocities ,filtered_velocities],
        labels=[['$v_x$', '$v_y$', '$v_z$'],
                ['$filtered v_x$', '$filtered v_y$', '$filtered v_z$']],
        show_legend=True)

In [ ]:
positions = []
p = np.array([[0, 0, 0]]).T

t = 0
while t < sample_number:
    at = filtered_a_nav[t, np.newaxis].T
    # vt = filtered_velocities[t, np.newaxis].T
    vt = velocities[t, np.newaxis].T

    p = p + vt * dt + 0.5 * at * dt**2
    positions.append(p.T[0])

    t += 1

positions = np.array(positions)
filtered_positions, = Filt_signal([positions], dt=dt, wn=1e-1, btype='highpass')

In [ ]:
plot_3D([[filtered_positions, 'position']])

In [ ]:
plt.close('all')